In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
### PART 1 ###
#Scraping the toronto burough info fro wikipedia
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 10 rows
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()[:-1]
    print (" %d : %s " % (i,name))
    col.append((name,[]))
col

 1 : Postal Code 
 2 : Borough 
 3 : Neighbourhood 


[('Postal Code', []), ('Borough', []), ('Neighbourhood', [])]

In [6]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()[:-1]#la parte de '[:,-1]'sirve para que no lea el /n de la tabla
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df=df.rename(columns={'Post Code':'Postal Code'})
df.tail()

,Postal Code,Borough,Neighbourhood
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned
180,,Canadian postal codes,


In [9]:
#exploring the data fram we see that last row is not data from the wikipedia table 
#and characters not desired at the end of each potition
df.drop(df.tail(1).index,inplace=True) # drop last n rows
#df.iloc[0,0]=str(df.iloc[0,0])
df.tail()

,PostalCode,Borough,Neighborhood
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned


In [10]:
'''I found a more optimal way to do it
#convert alll dataframe to string
df=df.astype(str)

#Function to remove last characters
def SuprLast2 (elemento):
    return elemento[:-1]
#remove /n
df.iloc[:,1]=df.iloc[:,1].map(SuprLast2)
df.iloc[:,0]=df.iloc[:,0].map(SuprLast2)
df.iloc[:,2]=df.iloc[:,2].map(SuprLast2)
df'''

'I found a more optimal way to do it\n#convert alll dataframe to string\ndf=df.astype(str)\n\n#Function to remove last characters\ndef SuprLast2 (elemento):\n    return elemento[:-1]\n#remove /n\ndf.iloc[:,1]=df.iloc[:,1].map(SuprLast2)\ndf.iloc[:,0]=df.iloc[:,0].map(SuprLast2)\ndf.iloc[:,2]=df.iloc[:,2].map(SuprLast2)\ndf'

In [11]:
#now should clean the df removing the not assigned elements
df=df[df.Borough != 'Not assigned']
df=df[df.Neighborhood != 'Not assigned']
#reset de index
df=df.reset_index(drop=True)

In [12]:
df.shape

(103, 3)

In [13]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [14]:
#botar el df en un archivo que pueda ser leido desde un archivo
df.to_csv('TorontoWiki.csv')